In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from Data_Processing import preprocessing, modify_target_binary
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, accuracy_score
from typing import Dict
from tqdm import tqdm
import numpy as np
from xgboost import XGBClassifier
from joblib import dump

In [7]:
loan_df = pd.read_csv("data/accepted_2007_to_2018Q4.csv")
print("DataFrame read!")
loan_df = modify_target_binary(loan_df, "loan_status")
# Remove Current/Issued targets before preprocessing
loan_df = loan_df.loc[~loan_df["loan_status"].isin(["Current","Issued"])].copy()

C:\Users\andre\AppData\Local\Temp\ipykernel_30928\2285443003.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_df = pd.read_csv("data/accepted_2007_to_2018Q4.csv")


DataFrame read!


Training using class weights and Random Grid Search

In [8]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state= 42)
RF_grid_dist =        {"n_estimators": [300],
                       "max_depth" :  np.arange(5, 61, 5),
                       "min_samples_split" : np.arange(2,21),
                       "max_features": ["sqrt","log2"],
                       "min_samples_leaf": np.arange(2,21),
                       "class_weight": ["balanced"],
                       "n_jobs" : [-1]
                       }

XGBoost_grid_dist = {"n_estimators": np.array([300]),
                     "max_depth": np.arange(5,61,5),
                     "learning_rate":np.array([0.001, 0.01, 0.1, 0.5]),
                     "objective": ["binary:logistic"],
                     "subsample": np.arange(0.1, 1.0, 0.1),
                     "colsample_bytree": np.arange(0.1, 1.0, 0.1),
                     "reg_alpha": np.array([0.01, 0.05, 0.1, 0.5, 0.8]),
                     "reg_lambda": np.array([0.01, 0.05, 0.1, 0.5, 0.8])}

# Function to perform random search with stratified k-fold validation
def RANDOM_SEARCH_CV_SS(param_dist: Dict, n_iterations: int, X_train: pd.DataFrame, y_train: pd.Series, model_classifier, cv:StratifiedKFold):
    results = []
    best_score = -np.inf
    best_params = None

    for _ in tqdm(range(n_iterations)):


        params = {}
        for key,values in param_dist.items():
            params[key] = np.random.choice(values)

        f1_scores = []
        recalls = []
        accuracy_scores = []

        for train_idx, val_idx in cv.split(X_train, y_train):
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            model = model_classifier(
                **params
            )

            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_val)

            f1_scores.append(f1_score(y_val, y_pred))
            recalls.append(recall_score(y_val, y_pred))
            accuracy_scores.append(accuracy_score(y_val, y_pred))

        mean_f1 = np.mean(f1_scores)

        if mean_f1 > best_score:
            best_score = mean_f1
            best_model = model
            best_params = params

    return best_params, best_score, best_model
    
# Function to tune random forest hyperparameters on sample of data
def hyperparameter_tuning(rows_sample:int, df:pd.DataFrame, cv:StratifiedKFold, param_dist:Dict, model_classifier):
    df_sample = df.sample(n = rows_sample)
    X_sample =  df_sample.drop(columns="loan_status")
    y_sample =  df_sample["loan_status"]
    X_sample = preprocessing(X_sample)
    best_params, best_score, best_model = RANDOM_SEARCH_CV_SS(param_dist, 50, X_sample, y_sample, model_classifier, cv)
    print(f"Best CV F1-score: {best_score}")
    print(best_params)
    return best_params, best_score, best_model

print("Random Forest: ")
best_RF_params, best_f1_score, best_RF_model = hyperparameter_tuning(10000, loan_df, skf, RF_grid_dist, RandomForestClassifier)
print("XGBoost :")
best_XGB_params, best_f1_score, best_XGB_model = hyperparameter_tuning(10000, loan_df, skf, XGBoost_grid_dist, XGBClassifier)

Random Forest: 


100%|██████████| 50/50 [05:16<00:00,  6.32s/it]


Best CV F1-score: 0.7016153391753142
{'n_estimators': np.int64(300), 'max_depth': np.int64(45), 'min_samples_split': np.int64(2), 'max_features': np.str_('sqrt'), 'min_samples_leaf': np.int64(3), 'class_weight': np.str_('balanced'), 'n_jobs': np.int64(-1)}
XGBoost :


100%|██████████| 50/50 [15:26<00:00, 18.52s/it]


Best CV F1-score: 0.739531258298104
{'n_estimators': np.int64(300), 'max_depth': np.int64(10), 'learning_rate': np.float64(0.1), 'objective': np.str_('binary:logistic'), 'subsample': np.float64(0.9), 'colsample_bytree': np.float64(0.5), 'reg_alpha': np.float64(0.8), 'reg_lambda': np.float64(0.05)}


In [ ]:
# Training using best hyperparameters on the whole dataset
X = loan_df.drop(columns=["loan_status"])
y = loan_df["loan_status"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, 
                                                    random_state = 42, 
                                                    shuffle = True, 
                                                    stratify = y)

X_train, X_test = preprocessing(X_train), preprocessing(X_test)
print("Processing finished!")
# XGBoost best params
XGB_model = XGBClassifier(**best_XGB_params)
XGB_model.fit(X_train,y_train)
print("XBG model trained!")
# Random Forest best params
RF_model = RandomForestClassifier(**best_RF_params)
RF_model.fit(X_train,y_train)
print("Random Forest trained!")

Processing finished!
XBG model trained!
Random Forest trained!


In [10]:
# Save the models trained with the best hyperparameters
dump(XGB_model, r"models\xgb_model_1.joblib")
dump(RF_model,  r"models\random_forest_model.joblib")
print("Models saved!")

Models saved!
